# Using Battery Parameter eXchange (BPX) files

In this notebook we how to import parameters defined using the [BPX standard](https://github.com/FaradayInstitution/BPX).

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm
import matplotlib.pyplot as plt
import numpy as np

First we load an example BPX file from the [pybamm-data](https://github.com/pybamm-team/pybamm-data/releases) repository using the `DataLoader` class. For more information on the `DataLoader` class, see the [pybamm_data](../pybamm_data.ipynb) notebook.

In [ ]:
path_to_bpx_file = pybamm.DataLoader().get_data("nmc_pouch_cell_BPX.json")

We can create a `ParameterValues` object from a BPX file using the `create_from_bpx` method. We pass in the path to the BPX file.

In [ ]:
parameter_values = pybamm.ParameterValues.create_from_bpx(path_to_bpx_file)

Alternatively, we can create a `ParameterValues` object from a BPX JSON object as `pybamm.ParameterValues.create_from_bpx_obj(bpx_obj)`


We can take a look at the parameter values we have loaded, and interact with them like any other `ParameterValues` object. For example, we can search for parameters containing the text "Negative electrode".

In [ ]:
parameter_values.search("Negative electrode")

Next we load a model and create a simulation, passing in our parameter values.

In [ ]:
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, parameter_values=parameter_values)

Finally, we can solve the model and plot the results.

In [ ]:
# solve for 1 hour
sim.solve([0, 3600])
_ = sim.plot()

We can also plot the parameters some of the parameters we loaded. Here we will plot the electrolyte conductivity and diffusivity, as well as the particle diffusivity, exchange-current density and open-circuit potential for the positive and negative electrodes over their respective stoichiometry ranges.

In [ ]:
# define stoichiometry ranges
xLi_n_max = parameter_values["Negative electrode maximum stoichiometry"]
xLi_n_min = parameter_values["Negative electrode minimum stoichiometry"]
xLi_n = pybamm.linspace(xLi_n_min, xLi_n_max, 200)
xLi_p_max = parameter_values["Positive electrode maximum stoichiometry"]
xLi_p_min = parameter_values["Positive electrode minimum stoichiometry"]
xLi_p = pybamm.linspace(xLi_p_min, xLi_p_max, 200)

# define concentration ranges
c_n_max = parameter_values["Maximum concentration in negative electrode [mol.m-3]"]
c_p_max = parameter_values["Maximum concentration in positive electrode [mol.m-3]"]
c_n = xLi_n * c_n_max
c_p = xLi_p * c_p_max
c_e = pybamm.linspace(0, 2000, 200)

c_e_ref = parameter_values["Initial concentration in electrolyte [mol.m-3]"]
T = parameter_values["Reference temperature [K]"]

# create figure
fig, ax = plt.subplots(4, 2, figsize=(16, 12))

ax[0, 0].plot(
    c_e.entries,
    parameter_values.evaluate(
        parameter_values["Electrolyte conductivity [S.m-1]"](c_e, T)
    ),
)
ax[0, 0].set(xlabel="$c_e$ [mol/m3]", ylabel="$\kappa$ [S.m-1]")
ax[0, 1].plot(
    c_e.entries,
    parameter_values.evaluate(
        parameter_values["Electrolyte diffusivity [m2.s-1]"](c_e, T)
    ),
)
ax[0, 1].set(xlabel="$c_e$ [mol/m3]", ylabel="$D_e$ [S.m-1]")
ax[1, 0].plot(
    xLi_n.entries,
    parameter_values.evaluate(
        parameter_values["Negative electrode diffusivity [m2.s-1]"](xLi_n, T)
    )
    * np.ones_like(xLi_n.entries),
)
ax[1, 0].set(xlabel="sto [-]", ylabel="$D_n$ [S.m-1]")
ax[1, 1].plot(
    xLi_p.entries,
    parameter_values.evaluate(
        parameter_values["Positive electrode diffusivity [m2.s-1]"](xLi_p, T)
    )
    * np.ones_like(xLi_p.entries),
)
ax[1, 1].set(xlabel="sto [-]", ylabel="$D_n$ [S.m-1]")
ax[2, 0].plot(
    xLi_n.entries,
    parameter_values.evaluate(
        parameter_values["Negative electrode exchange-current density [A.m-2]"](
            c_e_ref, c_n, c_n_max, T
        )
    ),
)
ax[2, 0].set(xlabel="sto [-]", ylabel="$j_n$ [A.m-2]")
ax[2, 1].plot(
    xLi_p.entries,
    parameter_values.evaluate(
        parameter_values["Positive electrode exchange-current density [A.m-2]"](
            c_e_ref, c_p, c_p_max, T
        )
    ),
)
ax[2, 1].set(xlabel="sto [-]", ylabel="$j_p$ [A.m-2]")
ax[3, 0].plot(
    xLi_n.entries, parameter_values["Negative electrode OCP [V]"](xLi_n).entries
)
ax[3, 0].set(xlabel="sto [-]", ylabel="$U_n$ [V]")
ax[3, 1].plot(
    xLi_p.entries, parameter_values["Positive electrode OCP [V]"](xLi_p).entries
)
ax[3, 1].set(xlabel="sto [-]", ylabel="$U_p$ [V]")

plt.tight_layout()
plt.show()

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()